<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/TextCNN_with_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/6b20ab3c-04a2-4b5b-b39b-6531835e3571

In [ ]:
pip install tensorflow

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
file_path = './saved_file'
file_path = ('ARPs_and_ProgrammingPosts.xlsx')

data = pd.read_excel(file_path)


# Preprocess the data

In [6]:
# Preprocess the data
X = data['Question_body']
y = data['Label']

In [7]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [9]:
# Parameters
vocab_size = 5000
embedding_dim = 100
max_length = 500
trunc_type = 'post'
padding_type = 'post'


In [10]:
# Tokenize the data
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [11]:
# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)


# TextCNN model

In [12]:
# TextCNN model
def create_text_cnn_model(vocab_size, embedding_dim, max_length):
    inputs = Input(shape=(max_length,))
    embedding = Embedding(vocab_size, embedding_dim, input_length=max_length)(inputs)

    conv1 = Conv1D(128, 3, activation='relu')(embedding)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(128, 4, activation='relu')(embedding)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(128, 5, activation='relu')(embedding)
    pool3 = MaxPooling1D(pool_size=2)(conv3)

    concatenated = concatenate([pool1, pool2, pool3], axis=1)
    flatten = Flatten()(concatenated)
    dense1 = Dense(128, activation='relu')(flatten)
    dropout = Dropout(0.5)(dense1)
    outputs = Dense(1, activation='sigmoid')(dropout)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create the model

In [13]:
# Create the model
text_cnn_model = create_text_cnn_model(vocab_size, embedding_dim, max_length)

# Display the model summary

In [ ]:
# Display the model summary
text_cnn_model.summary()

# Train the model

In [15]:
# Train the model
history = text_cnn_model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = text_cnn_model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
374/374 [==============================] - 254s 674ms/step - loss: 0.1704 - accuracy: 0.9421 - val_loss: 0.1088 - val_accuracy: 0.9672
Epoch 2/10
374/374 [==============================] - 249s 667ms/step - loss: 0.0693 - accuracy: 0.9802 - val_loss: 0.1205 - val_accuracy: 0.9672
Epoch 3/10
374/374 [==============================] - 250s 668ms/step - loss: 0.0389 - accuracy: 0.9877 - val_loss: 0.1599 - val_accuracy: 0.9675
Epoch 4/10
374/374 [==============================] - 251s 672ms/step - loss: 0.0239 - accuracy: 0.9910 - val_loss: 0.2187 - val_accuracy: 0.9648
Epoch 5/10
374/374 [==============================] - 251s 670ms/step - loss: 0.0120 - accuracy: 0.9956 - val_loss: 0.2961 - val_accuracy: 0.9602
Epoch 6/10
374/374 [==============================] - 252s 673ms/step - loss: 0.0251 - accuracy: 0.9931 - val_loss: 0.2787 - val_accuracy: 0.9588
Epoch 7/10
374/374 [==============================] - 242s 645ms/step - loss: 0.0134 - accuracy: 0.9972 - val_loss: 0.3696 -

# Predict on new data

In [16]:
# Predict on new data
y_pred = text_cnn_model.predict(X_test_padded)
y_pred_classes = (y_pred > 0.5).astype("int32")

94/94 [==============================] - 27s 287ms/step


# Evaluate predictions

In [17]:
# Evaluate predictions
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred_classes)
print(report)

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1504
           1       0.95      0.98      0.96      1483

    accuracy                           0.96      2987
   macro avg       0.96      0.96      0.96      2987
weighted avg       0.96      0.96      0.96      2987

